In [47]:
import torch
import torch_geometric as pyg
from sklearn.metrics import roc_auc_score, average_precision_score
from torch_geometric.nn import Node2Vec
from torch_geometric.transforms import RandomLinkSplit
from einops import rearrange

pyg.seed_everything(3407)
device = 'cuda:1' if torch.cuda.is_available() else 'cpu'

data = torch.load('/home/chenjunfen/workspace/XZH/graphemb/dydata/rc_snapshots/f1.pt')
data = data.to(device)
data['edge_index'] = torch.tensor(data['edge_index'])
data['edge_attr'] = torch.tensor(data['edge_attr'])
data['t_start'] = torch.tensor(data['t_start'])
data['num_nodes'] = torch.tensor(data['num_nodes'])
print(data)
transform = RandomLinkSplit(num_val=0.0,
                            num_test=0.1,
                            is_undirected=False)
train_data, val_data, test_data = transform(data)

model = Node2Vec(
    edge_index=train_data.edge_index,
    embedding_dim=128,
    walk_length=20,
    context_size=10,
    walks_per_node=10,
    num_negative_samples=1,
    p=1.0,  # p=q=1时，退化为DeepWalk
    q=0.5,
    sparse=True,  # 该数据集为稀疏数据集
).to(device)
loader = model.loader(batch_size=128, shuffle=True, num_workers=8)
optimizer = torch.optim.SparseAdam(list(model.parameters()), lr=0.01)
print(torch.max(train_data.edge_index[0]))
print(train_data.edge_index.shape)


def train():
    model.train()
    total_loss = 0
    for pos_rw, neg_rw in loader:
        optimizer.zero_grad()
        loss = model.loss(pos_rw.to(device), neg_rw.to(device))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)


@torch.no_grad()
def test():
    model.eval()
    data = test_data
    z = model()  # 输出潜在空间的嵌入向量
    out = (z[data.edge_label_index[0]] * z[data.edge_label_index[1]]).sum(dim=-1)
    out = out.view(-1).sigmoid()  # 重建edge_label与输入的边标签进行对比
    AUC = roc_auc_score(data.edge_label.cpu().numpy(), out.cpu().numpy())
    AP = average_precision_score(data.edge_label.cpu().numpy(), out.cpu().numpy())
    return AUC, AP


for epoch in range(1, 11):
    loss = train()
    AUC, AP = test()
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f},AUC: {AUC:.4f},AP: {AP:.4f}')

Data(edge_index=[2, 6243], edge_attr=[6243], t_start=0, num_nodes=6416)
tensor(26894)
torch.Size([2, 5619])
Epoch: 001, Loss: 4.2250,AUC: 0.7878,AP: 0.7406
Epoch: 002, Loss: 2.1525,AUC: 0.8903,AP: 0.8925
Epoch: 003, Loss: 1.2365,AUC: 0.9344,AP: 0.9536
Epoch: 004, Loss: 0.8903,AUC: 0.9448,AP: 0.9664
Epoch: 005, Loss: 0.7769,AUC: 0.9437,AP: 0.9674
Epoch: 006, Loss: 0.7438,AUC: 0.9430,AP: 0.9671
Epoch: 007, Loss: 0.7353,AUC: 0.9415,AP: 0.9665
Epoch: 008, Loss: 0.7334,AUC: 0.9443,AP: 0.9677
Epoch: 009, Loss: 0.7332,AUC: 0.9433,AP: 0.9673
Epoch: 010, Loss: 0.7330,AUC: 0.9433,AP: 0.9672


In [48]:
z1 = model()
z1.shape

torch.Size([26895, 128])

In [49]:
z=z1
q = 27045-z.shape[0]
zero = torch.zeros((q,128)).to(device)
z = torch.cat([z,zero],0)
z1=z
z1.shape

torch.Size([27045, 128])

In [50]:
import torch
import torch_geometric as pyg
from sklearn.metrics import roc_auc_score, average_precision_score
from torch_geometric.nn import Node2Vec
from torch_geometric.transforms import RandomLinkSplit
from einops import rearrange

pyg.seed_everything(3407)
device = 'cuda:1' if torch.cuda.is_available() else 'cpu'

data = torch.load('/home/chenjunfen/workspace/XZH/graphemb/dydata/rc_snapshots/f2.pt')
data = data.to(device)
data['edge_index'] = torch.tensor(data['edge_index'])
data['edge_attr'] = torch.tensor(data['edge_attr'])
data['t_start'] = torch.tensor(data['t_start'])
data['num_nodes'] = torch.tensor(data['num_nodes'])
print(data)
transform = RandomLinkSplit(num_val=0.0,
                            num_test=0.1,
                            is_undirected=False)
train_data, val_data, test_data = transform(data)

model = Node2Vec(
    edge_index=train_data.edge_index,
    embedding_dim=128,
    walk_length=20,
    context_size=10,
    walks_per_node=10,
    num_negative_samples=1,
    p=1.0,  # p=q=1时，退化为DeepWalk
    q=0.1,
    sparse=True,  # 该数据集为稀疏数据集
).to(device)
loader = model.loader(batch_size=128, shuffle=True, num_workers=4)
optimizer = torch.optim.SparseAdam(list(model.parameters()), lr=0.01)
print(torch.max(train_data.edge_index[0]))
print(train_data.edge_index.shape)


def train():
    model.train()
    total_loss = 0
    for pos_rw, neg_rw in loader:
        optimizer.zero_grad()
        loss = model.loss(pos_rw.to(device), neg_rw.to(device))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)


@torch.no_grad()
def test():
    model.eval()
    data = test_data
    z = model()  # 输出潜在空间的嵌入向量
    out = (z[data.edge_label_index[0]] * z[data.edge_label_index[1]]).sum(dim=-1)
    out = out.view(-1).sigmoid()  # 重建edge_label与输入的边标签进行对比
    AUC = roc_auc_score(data.edge_label.cpu().numpy(), out.cpu().numpy())
    AP = average_precision_score(data.edge_label.cpu().numpy(), out.cpu().numpy())
    return AUC, AP


for epoch in range(1, 11):
    loss = train()
    AUC, AP = test()
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f},AUC: {AUC:.4f},AP: {AP:.4f}')

Data(edge_index=[2, 7030], edge_attr=[7030], t_start=6243, num_nodes=6416)
tensor(26903)
torch.Size([2, 6327])
Epoch: 001, Loss: 4.2303,AUC: 0.7941,AP: 0.7274
Epoch: 002, Loss: 2.1544,AUC: 0.8972,AP: 0.8914
Epoch: 003, Loss: 1.2373,AUC: 0.9450,AP: 0.9608
Epoch: 004, Loss: 0.8904,AUC: 0.9533,AP: 0.9712
Epoch: 005, Loss: 0.7772,AUC: 0.9541,AP: 0.9726
Epoch: 006, Loss: 0.7444,AUC: 0.9543,AP: 0.9724
Epoch: 007, Loss: 0.7359,AUC: 0.9550,AP: 0.9729
Epoch: 008, Loss: 0.7340,AUC: 0.9531,AP: 0.9725
Epoch: 009, Loss: 0.7336,AUC: 0.9529,AP: 0.9721
Epoch: 010, Loss: 0.7334,AUC: 0.9530,AP: 0.9722


In [51]:
z2 = model()
z2.shape

torch.Size([26904, 128])

In [52]:
z=z2
q = 27045-z.shape[0]
zero = torch.zeros((q,128)).to(device)
z = torch.cat([z,zero],0)
z2=z
z2.shape

torch.Size([27045, 128])

In [53]:
import torch
import torch_geometric as pyg
from sklearn.metrics import roc_auc_score, average_precision_score
from torch_geometric.nn import Node2Vec
from torch_geometric.transforms import RandomLinkSplit
from einops import rearrange

pyg.seed_everything(3407)
device = 'cuda:1' if torch.cuda.is_available() else 'cpu'

data = torch.load('/home/chenjunfen/workspace/XZH/graphemb/dydata/rc_snapshots/f3.pt')
data = data.to(device)
data['edge_index'] = torch.tensor(data['edge_index'])
data['edge_attr'] = torch.tensor(data['edge_attr'])
data['t_start'] = torch.tensor(data['t_start'])
data['num_nodes'] = torch.tensor(data['num_nodes'])
print(data)
transform = RandomLinkSplit(num_val=0.0,
                            num_test=0.1,
                            is_undirected=False)
train_data, val_data, test_data = transform(data)

model = Node2Vec(
    edge_index=train_data.edge_index,
    embedding_dim=128,
    walk_length=20,
    context_size=10,
    walks_per_node=10,
    num_negative_samples=1,
    p=1.0,  # p=q=1时，退化为DeepWalk
    q=0.1,
    sparse=True,  # 该数据集为稀疏数据集
).to(device)
loader = model.loader(batch_size=128, shuffle=True, num_workers=4)
optimizer = torch.optim.SparseAdam(list(model.parameters()), lr=0.01)
print(torch.max(train_data.edge_index[0]))
print(train_data.edge_index.shape)


def train():
    model.train()
    total_loss = 0
    for pos_rw, neg_rw in loader:
        optimizer.zero_grad()
        loss = model.loss(pos_rw.to(device), neg_rw.to(device))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)


@torch.no_grad()
def test():
    model.eval()
    data = test_data
    z = model()  # 输出潜在空间的嵌入向量
    out = (z[data.edge_label_index[0]] * z[data.edge_label_index[1]]).sum(dim=-1)
    out = out.view(-1).sigmoid()  # 重建edge_label与输入的边标签进行对比
    AUC = roc_auc_score(data.edge_label.cpu().numpy(), out.cpu().numpy())
    AP = average_precision_score(data.edge_label.cpu().numpy(), out.cpu().numpy())
    return AUC, AP


for epoch in range(1, 11):
    loss = train()
    AUC, AP = test()
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f},AUC: {AUC:.4f},AP: {AP:.4f}')

Data(edge_index=[2, 9081], edge_attr=[9081], t_start=13273, num_nodes=6416)
tensor(26976)
torch.Size([2, 8173])
Epoch: 001, Loss: 4.2391,AUC: 0.7636,AP: 0.7158
Epoch: 002, Loss: 2.1601,AUC: 0.8678,AP: 0.8716
Epoch: 003, Loss: 1.2388,AUC: 0.9202,AP: 0.9473
Epoch: 004, Loss: 0.8913,AUC: 0.9312,AP: 0.9602
Epoch: 005, Loss: 0.7782,AUC: 0.9320,AP: 0.9622
Epoch: 006, Loss: 0.7454,AUC: 0.9328,AP: 0.9629
Epoch: 007, Loss: 0.7371,AUC: 0.9328,AP: 0.9627
Epoch: 008, Loss: 0.7350,AUC: 0.9338,AP: 0.9632
Epoch: 009, Loss: 0.7341,AUC: 0.9369,AP: 0.9648
Epoch: 010, Loss: 0.7337,AUC: 0.9372,AP: 0.9646


In [54]:
z3 = model()
z3.shape

torch.Size([26986, 128])

In [55]:
z=z3
q = 27045-z.shape[0]
zero = torch.zeros((q,128)).to(device)
z = torch.cat([z,zero],0)
z3=z
z3.shape

torch.Size([27045, 128])

In [56]:
import torch
import torch_geometric as pyg
from sklearn.metrics import roc_auc_score, average_precision_score
from torch_geometric.nn import Node2Vec
from torch_geometric.transforms import RandomLinkSplit
from einops import rearrange

pyg.seed_everything(3407)
device = 'cuda:1' if torch.cuda.is_available() else 'cpu'

data = torch.load('/home/chenjunfen/workspace/XZH/graphemb/dydata/rc_snapshots/f4.pt')
data = data.to(device)
data['edge_index'] = torch.tensor(data['edge_index'])
data['edge_attr'] = torch.tensor(data['edge_attr'])
data['t_start'] = torch.tensor(data['t_start'])
data['num_nodes'] = torch.tensor(data['num_nodes'])
print(data)
transform = RandomLinkSplit(num_val=0.0,
                            num_test=0.1,
                            is_undirected=False)
train_data, val_data, test_data = transform(data)

model = Node2Vec(
    edge_index=train_data.edge_index,
    embedding_dim=128,
    walk_length=20,
    context_size=10,
    walks_per_node=10,
    num_negative_samples=1,
    p=1.0,  # p=q=1时，退化为DeepWalk
    q=0.1,
    sparse=True,  # 该数据集为稀疏数据集
).to(device)
loader = model.loader(batch_size=128, shuffle=True, num_workers=4)
optimizer = torch.optim.SparseAdam(list(model.parameters()), lr=0.01)
print(torch.max(train_data.edge_index[0]))
print(train_data.edge_index.shape)


def train():
    model.train()
    total_loss = 0
    for pos_rw, neg_rw in loader:
        optimizer.zero_grad()
        loss = model.loss(pos_rw.to(device), neg_rw.to(device))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)


@torch.no_grad()
def test():
    model.eval()
    data = test_data
    z = model()  # 输出潜在空间的嵌入向量
    out = (z[data.edge_label_index[0]] * z[data.edge_label_index[1]]).sum(dim=-1)
    out = out.view(-1).sigmoid()  # 重建edge_label与输入的边标签进行对比
    AUC = roc_auc_score(data.edge_label.cpu().numpy(), out.cpu().numpy())
    AP = average_precision_score(data.edge_label.cpu().numpy(), out.cpu().numpy())
    return AUC, AP


for epoch in range(1, 11):
    loss = train()
    AUC, AP = test()
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f},AUC: {AUC:.4f},AP: {AP:.4f}')

Data(edge_index=[2, 7642], edge_attr=[7642], t_start=22354, num_nodes=6416)
tensor(26956)
torch.Size([2, 6878])
Epoch: 001, Loss: 4.2371,AUC: 0.7578,AP: 0.6934
Epoch: 002, Loss: 2.1580,AUC: 0.8581,AP: 0.8565
Epoch: 003, Loss: 1.2373,AUC: 0.9113,AP: 0.9387
Epoch: 004, Loss: 0.8899,AUC: 0.9235,AP: 0.9542
Epoch: 005, Loss: 0.7772,AUC: 0.9260,AP: 0.9574
Epoch: 006, Loss: 0.7446,AUC: 0.9277,AP: 0.9582
Epoch: 007, Loss: 0.7363,AUC: 0.9252,AP: 0.9572
Epoch: 008, Loss: 0.7345,AUC: 0.9249,AP: 0.9565
Epoch: 009, Loss: 0.7338,AUC: 0.9272,AP: 0.9578
Epoch: 010, Loss: 0.7336,AUC: 0.9283,AP: 0.9581


In [57]:
z4 = model()
z4.shape

torch.Size([26958, 128])

In [58]:
z=z4
q = 27045-z.shape[0]
zero = torch.zeros((q,128)).to(device)
z = torch.cat([z,zero],0)
z4=z
z4.shape

torch.Size([27045, 128])

In [61]:
import torch
import torch_geometric as pyg
from sklearn.metrics import roc_auc_score, average_precision_score
from torch_geometric.nn import Node2Vec
from torch_geometric.transforms import RandomLinkSplit
from einops import rearrange

pyg.seed_everything(3407)
device = 'cuda:1' if torch.cuda.is_available() else 'cpu'

data = torch.load('/home/chenjunfen/workspace/XZH/graphemb/dydata/rc_snapshots/f5.pt')
data = data.to(device)
data['edge_index'] = torch.tensor(data['edge_index'])
data['edge_attr'] = torch.tensor(data['edge_attr'])
data['t_start'] = torch.tensor(data['t_start'])
data['num_nodes'] = torch.tensor(data['num_nodes'])
print(data)
transform = RandomLinkSplit(num_val=0.0,
                            num_test=0.1,
                            is_undirected=False)
train_data, val_data, test_data = transform(data)

model = Node2Vec(
    edge_index=train_data.edge_index,
    embedding_dim=128,
    walk_length=20,
    context_size=10,
    walks_per_node=10,
    num_negative_samples=1,
    p=1.0,  # p=q=1时，退化为DeepWalk
    q=0.1,
    sparse=True,  # 该数据集为稀疏数据集
).to(device)
loader = model.loader(batch_size=128, shuffle=True, num_workers=4)
optimizer = torch.optim.SparseAdam(list(model.parameters()), lr=0.01)
print(torch.max(train_data.edge_index[0]))
print(train_data.edge_index.shape)


def train():
    model.train()
    total_loss = 0
    for pos_rw, neg_rw in loader:
        optimizer.zero_grad()
        loss = model.loss(pos_rw.to(device), neg_rw.to(device))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)


@torch.no_grad()
def test():
    model.eval()
    data = test_data
    z = model()  # 输出潜在空间的嵌入向量
    out = (z[data.edge_label_index[0]] * z[data.edge_label_index[1]]).sum(dim=-1)
    out = out.view(-1).sigmoid()  # 重建edge_label与输入的边标签进行对比
    AUC = roc_auc_score(data.edge_label.cpu().numpy(), out.cpu().numpy())
    AP = average_precision_score(data.edge_label.cpu().numpy(), out.cpu().numpy())
    return AUC, AP


for epoch in range(1, 11):
    loss = train()
    AUC, AP = test()
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f},AUC: {AUC:.4f},AP: {AP:.4f}')

Data(edge_index=[2, 7802], edge_attr=[7802], t_start=29996, num_nodes=6416)
tensor(27005)
torch.Size([2, 7022])
Epoch: 001, Loss: 4.2330,AUC: 0.7564,AP: 0.6929
Epoch: 002, Loss: 2.1557,AUC: 0.8650,AP: 0.8639
Epoch: 003, Loss: 1.2403,AUC: 0.9155,AP: 0.9403
Epoch: 004, Loss: 0.8936,AUC: 0.9307,AP: 0.9592
Epoch: 005, Loss: 0.7811,AUC: 0.9332,AP: 0.9625
Epoch: 006, Loss: 0.7480,AUC: 0.9320,AP: 0.9623
Epoch: 007, Loss: 0.7395,AUC: 0.9287,AP: 0.9603
Epoch: 008, Loss: 0.7373,AUC: 0.9324,AP: 0.9616
Epoch: 009, Loss: 0.7365,AUC: 0.9369,AP: 0.9637
Epoch: 010, Loss: 0.7357,AUC: 0.9362,AP: 0.9632


In [62]:
z5 = model()
z5.shape

torch.Size([27009, 128])

In [63]:
z=z5
q = 27045-z.shape[0]
zero = torch.zeros((q,128)).to(device)
z = torch.cat([z,zero],0)
z5=z
z5.shape

torch.Size([27045, 128])

In [64]:
import torch
import torch_geometric as pyg
from sklearn.metrics import roc_auc_score, average_precision_score
from torch_geometric.nn import Node2Vec
from torch_geometric.transforms import RandomLinkSplit
from einops import rearrange

pyg.seed_everything(3407)
device = 'cuda:1' if torch.cuda.is_available() else 'cpu'

data = torch.load('/home/chenjunfen/workspace/XZH/graphemb/dydata/rc_snapshots/f6.pt')
data = data.to(device)
data['edge_index'] = torch.tensor(data['edge_index'])
data['edge_attr'] = torch.tensor(data['edge_attr'])
data['t_start'] = torch.tensor(data['t_start'])
data['num_nodes'] = torch.tensor(data['num_nodes'])
print(data)
transform = RandomLinkSplit(num_val=0.0,
                            num_test=50,
                            is_undirected=False)
train_data, val_data, test_data = transform(data)

model = Node2Vec(
    edge_index=train_data.edge_index,
    embedding_dim=128,
    walk_length=20,
    context_size=10,
    walks_per_node=10,
    num_negative_samples=1,
    p=1.0,  # p=q=1时，退化为DeepWalk
    q=0.1,
    sparse=True,  # 该数据集为稀疏数据集
).to(device)
loader = model.loader(batch_size=128, shuffle=True, num_workers=5)
optimizer = torch.optim.SparseAdam(list(model.parameters()), lr=0.01)
print(torch.max(train_data.edge_index[0]))
print(train_data.edge_index.shape)


def train():
    model.train()
    total_loss = 0
    for pos_rw, neg_rw in loader:
        optimizer.zero_grad()
        loss = model.loss(pos_rw.to(device), neg_rw.to(device))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)


@torch.no_grad()
def test():
    model.eval()
    data = test_data
    z = model()  # 输出潜在空间的嵌入向量
    out = (z[data.edge_label_index[0]] * z[data.edge_label_index[1]]).sum(dim=-1)
    out = out.view(-1).sigmoid()  # 重建edge_label与输入的边标签进行对比
    AUC = roc_auc_score(data.edge_label.cpu().numpy(), out.cpu().numpy())
    AP = average_precision_score(data.edge_label.cpu().numpy(), out.cpu().numpy())
    return AUC, AP


for epoch in range(1, 11):
    loss = train()
    AUC, AP = test()
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f},AUC: {AUC:.4f},AP: {AP:.4f}')

Data(edge_index=[2, 5714], edge_attr=[5714], t_start=37798, num_nodes=6416)
tensor(27011)
torch.Size([2, 5664])
Epoch: 001, Loss: 4.2314,AUC: 0.7984,AP: 0.7936
Epoch: 002, Loss: 2.1549,AUC: 0.8668,AP: 0.8929
Epoch: 003, Loss: 1.2366,AUC: 0.8932,AP: 0.9350
Epoch: 004, Loss: 0.8898,AUC: 0.8968,AP: 0.9416
Epoch: 005, Loss: 0.7768,AUC: 0.8924,AP: 0.9414
Epoch: 006, Loss: 0.7444,AUC: 0.8984,AP: 0.9437
Epoch: 007, Loss: 0.7358,AUC: 0.8972,AP: 0.9423
Epoch: 008, Loss: 0.7340,AUC: 0.9068,AP: 0.9464
Epoch: 009, Loss: 0.7336,AUC: 0.8996,AP: 0.9401
Epoch: 010, Loss: 0.7335,AUC: 0.9036,AP: 0.9406


In [65]:
z6 = model()
z6.shape

torch.Size([27027, 128])

In [66]:
z=z6
q = 27045-z.shape[0]
zero = torch.zeros((q,128)).to(device)
z = torch.cat([z,zero],0)
z6=z
z6.shape

torch.Size([27045, 128])

In [67]:
z_1_6 = torch.cat([z1,z2,z3,z4,z5,z6],1)
z_1_6.shape

torch.Size([27045, 768])

In [1]:
import torch
import torch_geometric as pyg
from sklearn.metrics import roc_auc_score, average_precision_score
from torch_geometric.nn import Node2Vec
from torch_geometric.transforms import RandomLinkSplit
from einops import rearrange

pyg.seed_everything(3407)
device = 'cuda:1' if torch.cuda.is_available() else 'cpu'

data = torch.load('/home/chenjunfen/workspace/XZH/graphemb/dydata/rc_snapshots/f7.pt')
data = data.to(device)
data['edge_index'] = torch.tensor(data['edge_index'])
data['edge_attr'] = torch.tensor(data['edge_attr'])
data['t_start'] = torch.tensor(data['t_start'])
data['num_nodes'] = torch.tensor(data['num_nodes'])
print(data)
transform = RandomLinkSplit(num_val=0.0,
                            num_test=50,
                            is_undirected=False)
train_data, val_data, test_data = transform(data)

model = Node2Vec(
    edge_index=train_data.edge_index,
    embedding_dim=128,
    walk_length=20,
    context_size=10,
    walks_per_node=10,
    num_negative_samples=1,
    p=1.0,  # p=q=1时，退化为DeepWalk
    q=0.1,
    sparse=True,  # 该数据集为稀疏数据集
).to(device)
loader = model.loader(batch_size=128, shuffle=True, num_workers=5)
optimizer = torch.optim.SparseAdam(list(model.parameters()), lr=0.01)
print(torch.max(train_data.edge_index[0]))
print(train_data.edge_index.shape)


def train():
    model.train()
    total_loss = 0
    for pos_rw, neg_rw in loader:
        optimizer.zero_grad()
        loss = model.loss(pos_rw.to(device), neg_rw.to(device))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)


@torch.no_grad()
def test():
    model.eval()
    data = test_data
    z = model()  # 输出潜在空间的嵌入向量
    out = (z[data.edge_label_index[0]] * z[data.edge_label_index[1]]).sum(dim=-1)
    out = out.view(-1).sigmoid()  # 重建edge_label与输入的边标签进行对比
    AUC = roc_auc_score(data.edge_label.cpu().numpy(), out.cpu().numpy())
    AP = average_precision_score(data.edge_label.cpu().numpy(), out.cpu().numpy())
    return AUC, AP


for epoch in range(1, 2):
    loss = train()
    AUC, AP = test()
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f},AUC: {AUC:.4f},AP: {AP:.4f}')

Data(edge_index=[2, 8538], edge_attr=[8538], t_start=43512, num_nodes=6416)
tensor(27043)
torch.Size([2, 8488])
Epoch: 001, Loss: 4.2370,AUC: 0.7878,AP: 0.7363
Epoch: 002, Loss: 2.1569,AUC: 0.9020,AP: 0.9060
Epoch: 003, Loss: 1.2380,AUC: 0.9288,AP: 0.9553
Epoch: 004, Loss: 0.8910,AUC: 0.9376,AP: 0.9638
Epoch: 005, Loss: 0.7778,AUC: 0.9352,AP: 0.9643
Epoch: 006, Loss: 0.7452,AUC: 0.9364,AP: 0.9659
Epoch: 007, Loss: 0.7369,AUC: 0.9356,AP: 0.9650
Epoch: 008, Loss: 0.7348,AUC: 0.9324,AP: 0.9632
Epoch: 009, Loss: 0.7341,AUC: 0.9448,AP: 0.9677
Epoch: 010, Loss: 0.7337,AUC: 0.9636,AP: 0.9779


In [2]:
z7 = model()
z7.shape

torch.Size([27045, 128])

In [22]:
# torch.save(z_1_6,'/home/chenjunfen/workspace/XZH/graphemb/dydata/rc_snapshots/z_1_6.pt')

In [74]:
import torch
import torch_geometric as pyg
from sklearn.metrics import roc_auc_score, average_precision_score
from torch_geometric.nn import Node2Vec
from torch_geometric.transforms import RandomLinkSplit
z_1_6=torch.load('/home/chenjunfen/workspace/XZH/graphemb/dydata/rc_snapshots/z_1_6.pt')
pyg.seed_everything(3407)
device = 'cuda:1' if torch.cuda.is_available() else 'cpu'

data = torch.load('/home/chenjunfen/workspace/XZH/graphemb/dydata/rc_snapshots/f7.pt')
data = data.to(device)
data['edge_index'] = torch.tensor(data['edge_index'])
data['edge_attr'] = torch.tensor(data['edge_attr'])
data['t_start'] = torch.tensor(data['t_start'])
data['num_nodes'] = torch.tensor(data['num_nodes'])

transform = RandomLinkSplit(num_val=0.0,
                            num_test=0.0,
                            is_undirected=False)
train_data, val_data, test_data = transform(data)

print(train_data.edge_index.shape)

@torch.no_grad()
def test():
    data = train_data
    z = z_1_6  # 输出潜在空间的嵌入向量
    out = (z[data.edge_label_index[0]] * z[data.edge_label_index[1]]).sum(dim=-1)
    out = out.view(-1).sigmoid()  # 重建edge_label与输入的边标签进行对比
    AUC = roc_auc_score(data.edge_label.cpu().numpy(), out.cpu().numpy())
    AP = average_precision_score(data.edge_label.cpu().numpy(), out.cpu().numpy())
    return AUC, AP


AUC, AP = test()
print(f'AUC: {AUC:.4f},AP: {AP:.4f}')

torch.Size([2, 8538])
AUC: 0.8704,AP: 0.9207
